<a href="https://colab.research.google.com/github/kingsdigitallab/lwm-davizct/blob/main/notebooks/Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Export data for _frontend_ visualisations
## Instructions

- [Download data from Zooniverse](https://github.com/Living-with-machines/zoonyper/blob/v0.1.0/docs/source/getting-started/tutorials/setting-up-your-first-project.rst)
- Set up the data path settings
- Run the rest of the notebook


## Settings

In [ ]:
ipython = get_ipython()
IN_COLAB = "google.colab" in str(ipython)

In [1]:
in_path = "../data/0_raw"
ext_path = "../data/0_external"
downloads_path = "../data/downloads"
out_path = "../data/1_interim"

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")
    in_path = "drive/My Drive/Colab Data/lwm/davizct/data/0_raw"
    ext_path = "drive/My Drive/Colab Data/lwm/davizct/data/0_external"
    downloads_path = "drive/My Drive/Colab Data/lwm/davizct/data/downloads"
    out_path = "drive/My Drive/Colab Data/lwm/davizct/data/1_interim"

## System set up

In [2]:
import base64
import json
import os
import re
import time
import zlib
from collections import Counter
from pprint import pprint

import pandas as pd

from zoonyper.project import Project

In [3]:
project = Project(in_path)
subjects_df = project.disambiguate_subjects(downloads_path)

⚠ Warning: Note that the subject IDs have not yet been
disambiguated. If you want to do so, run the
`.disambiguate_subjects(<download-dir>)` method.


100%|██████████████████████████████████████████████████████████████████████████████████| 51489/51489 [00:12<00:00, 4065.61it/s]


## Workflows

In [4]:
stat = os.stat(os.path.join(in_path, "workflows.csv"))
export_date = time.ctime(stat.st_mtime)

with open(os.path.join(out_path, "date.json"), "w") as f:
    json.dump(dict(date=export_date), f)

In [5]:
workflows_df = project.workflows

projects = [
    dict(
        title="Language of accidents",
        workflows=[
            dict(
                workflow_id=18831, name=workflows_df.loc[18831]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=20921, name=workflows_df.loc[20921]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=21525, name=workflows_df.loc[21525]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=23457, name=workflows_df.loc[23457]["display_name"].iloc[0]
            ),
        ],
    ),
    dict(
        title="Language of mechanisation",
        workflows=[
            dict(
                workflow_id=23672, name=workflows_df.loc[23672]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=23681, name=workflows_df.loc[23681]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=23628, name=workflows_df.loc[23628]["display_name"].iloc[0]
            ),
            dict(
                workflow_id=23452, name=workflows_df.loc[23452]["display_name"].iloc[0]
            ),
        ],
    ),
]

projects_workflows = [w["workflow_id"] for p in projects for w in p["workflows"]]
pprint(projects)

with open(os.path.join(out_path, "projects.json"), "w") as f:
    json.dump(projects, f)

[{'title': 'Language of accidents',
  'workflows': [{'name': 'Accident detail: age and gender of victims',
                 'workflow_id': 18831},
                {'name': 'How did machines change accidents?',
                 'workflow_id': 20921},
                {'name': 'Accident detail: in which sites were accidents '
                         'reported?',
                 'workflow_id': 21525},
                {'name': 'Accident detail: where were accidents reported?',
                 'workflow_id': 23457}]},
 {'title': 'Language of mechanisation',
  'workflows': [{'name': 'Bicycle or motorcycle?', 'workflow_id': 23672},
                {'name': "How did the word 'coach' change over time and place?",
                 'workflow_id': 23681},
                {'name': "How did the word 'car' change over time and place?",
                 'workflow_id': 23628},
                {'name': "How did the word 'trolley' change over time and "
                         'place?',
              

In [6]:
timelines = project.get_workflow_timelines()


def get_date(workflow_id, date_field):
    found = [t for t in timelines if t["workflow_id"] == workflow_id]
    if found:
        return found[0][date_field]

    return None


def parse_tasks(row):
    first = row["first_task"]
    tasks = json.loads(row["tasks"])

    return [k for k, v in tasks.items() if v["type"] != "drawing"]


workflows_df = project.workflows
workflows_df = workflows_df[~workflows_df.index.duplicated(keep="last")]
workflows_df = workflows_df.drop(
    columns=[
        "pairwise",
        "grouped",
        "prioritized",
        "primary_language",
        "tutorial_subject_id",
        "retired_set_member_subjects_count",
        "retirement",
        "aggregation",
        "strings",
    ]
)
workflows_df = workflows_df[workflows_df.index.isin(projects_workflows)]
workflows_df["start_date"] = workflows_df.index.map(lambda x: get_date(x, "start_date"))
workflows_df["end_date"] = workflows_df.index.map(lambda x: get_date(x, "end_date"))
workflows_df["tasks"] = workflows_df.apply(parse_tasks, axis=1)
workflows_df.to_json(os.path.join(out_path, "workflows.json"), orient="index")
workflows_df.head(10)

,display_name,version,active,classifications_count,first_task,tasks,minor_version,start_date,end_date
workflow_id,,,,,,,,,
18831,Accident detail: age and gender of victims,52,False,16715,T2,"[T1, T2, T3, T4, T6]",157,2021-06-21,2023-06-03
20921,How did machines change accidents?,10,False,76742,T0,[T0],81,2022-02-16,2023-06-07
21525,Accident detail: in which sites were accidents...,20,False,24633,T0,"[T0, T1]",56,2022-05-25,2023-06-03
23452,How did the word 'trolley' change over time an...,26,False,3346,T0,[T0],77,2023-02-06,2023-03-19
23457,Accident detail: where were accidents reported?,13,False,18426,T1,"[T0, T1, T2, T3]",54,2023-02-05,2023-06-04
23628,How did the word 'car' change over time and pl...,13,False,6365,T0,[T0],60,2023-02-27,2023-05-16
23672,Bicycle or motorcycle?,3,False,7731,T0,[T0],10,2023-03-07,2023-03-19
23681,How did the word 'coach' change over time and ...,8,False,6479,T0,[T0],55,2023-03-23,2023-05-15


## Participants

In [7]:
participants_df = project.classifications[
    ["workflow_id", "user_name", "user_logged_in"]
]
participants_df = participants_df[
    participants_df["workflow_id"].isin(projects_workflows)
]
participants_df = participants_df.value_counts(
    ["workflow_id", "user_name", "user_logged_in"]
).reset_index(name="count")
participants_df = participants_df.sort_values(by="workflow_id")

participants_df.to_json(os.path.join(out_path, "participants.json"), orient="records")

participants_df.head()

,workflow_id,user_name,user_logged_in,count
3132,18831,7a36deef,False,2
1084,18831,bb7798ec,True,16
2268,18831,be1d654b,False,5
2269,18831,a82e54f6,False,5
2270,18831,3f0c440f,True,5


## Subjects

In [8]:
def get_subject_date(row):
    if row["newspaper date"]:
        return row["newspaper date"]
    if row["newspaper_date"]:
        return row["newspaper_date"]
    if row["!original_metadata.newspaper date"]:
        return row["!original_metadata.newspaper date"]


subjects = subjects_df.reset_index()

subjects = subjects[subjects["workflow_id"].isin(projects_workflows)]
subjects = subjects[subjects["classifications_count"] > 0]
subjects = subjects.rename(
    columns={
        "subject_id_disambiguated": "sid",
        "classifications_count": "cc",
        "retirement_reason": "rr",
        "!inferred_nlp": "nlp",
        "!text": "text",
    }
)
subjects["sid"] = pd.to_numeric(subjects["sid"]).astype("Int64")
subjects["nlp"] = pd.to_numeric(subjects["nlp"], errors="coerce").astype("Int64")
subjects["nlp"] = subjects["nlp"].fillna(0)
subjects["img"] = subjects["locations"].apply(lambda x: list(x.values())[-1])
subjects["year"] = subjects.apply(get_subject_date, axis=1)
subjects["year"] = subjects["year"].apply(lambda x: x.split("-")[0])
subjects["t"] = subjects["text"].apply(
    lambda x: x.split(".")[0].strip()[:50] if x else ""
)

subjects = subjects[
    subjects.columns.drop(list(subjects.filter(regex="!original_metadata")))
]
subjects = subjects[subjects.columns.drop(list(subjects.filter(regex="newspaper")))]
subjects = subjects.drop(
    columns=[
        "!article_id",
        "!crop_rectangle",
        "!image",
        "!issue",
        "!myid",
        "!origin",
        "!original_id",
        "!page",
        "!text_source",
        "!textblock_id",
        "!zooniverse_file_md5",
        "!zooniverse_file_stem",
        "attribution",
        "locations",
        "subject_set_id",
        "origin",
        "project_id",
        "",
    ]
)
subjects = subjects.sort_values(by="sid")

# subject id disambuguated/subject ids links
subjects[["sid", "subject_id"]].drop_duplicates().set_index("subject_id").to_json(
    os.path.join(out_path, "subjects_ids.json"), orient="index"
)

# subjects metadata
subjects.drop_duplicates(subset=["sid", "workflow_id"]).drop(
    columns=["subject_id", "retired_at", "created_at", "updated_at", "text", "img"]
).to_json(os.path.join(out_path, "subjects.json"), orient="records")

# subjects texts, compressed
subjects_text = subjects[["sid", "text"]].drop_duplicates(subset="sid")
subjects_text.set_index("sid")
subjects_text["text"] = (
    subjects_text["text"]
    .fillna("")
    .apply(lambda x: base64.b64encode(zlib.compress(x.encode("utf-8"))))
)
subjects_text.to_json(os.path.join(out_path, "subjects_text.json"), orient="index")

# subjects images
subjects[["sid", "img"]].drop_duplicates(subset="sid").set_index("sid").to_json(
    os.path.join(out_path, "subjects_image.json"), orient="index"
)

subjects.head()

,subject_id,sid,workflow_id,cc,retired_at,rr,created_at,updated_at,nlp,text,img,year,t
214974,82929646,1,20921,3,,,2022-11-23,2023-04-02,2596,"_ Mir L AcctnrNY.—Priscilla Riley, a settor-io...",https://panoptes-uploads.zooniverse.org/subjec...,1889,_ Mir L AcctnrNY
287653,86518535,2,23681,3,2023-03-31,classification_count,2023-03-22,2023-04-01,3075,ortune in fuer seet — t — a:coinpativing indus...,https://panoptes-uploads.zooniverse.org/subjec...,1895,ortune in fuer seet — t — a:coinpativing indus...
217732,82929991,3,20921,4,2023-01-28,consensus,2022-11-23,2023-04-01,3089,"Flats.—On Saturdav night, shortly after ten o'...",https://panoptes-uploads.zooniverse.org/subjec...,1854,Flats
248941,82934613,4,20921,4,2023-05-15,classification_count,2022-11-23,2023-03-31,2596,HIGGINSHAW. STALYBRIDGE. STOPPAGE.—SeveraI pai...,https://panoptes-uploads.zooniverse.org/subjec...,1911,HIGGINSHAW
140275,82894129,5,20921,4,2022-12-06,consensus,2022-11-22,2023-04-02,2597,aREENHOUSES anti - glass 10 ft. by 5 ft. £6. ‘...,https://panoptes-uploads.zooniverse.org/subjec...,1874,aREENHOUSES anti - glass 10 ft


## Annotations

In [9]:
all_df = pd.read_csv(os.path.join(ext_path, 'all_accidents_all_data.tsv'), sep='\t')
all_df.columns

Index(['classification_id', 'workflow_id', 'created_at', 'user_hash',
       'newspaper place', 'newspaper title', 'article date', 'nlp',
       'image_url', 'metadata_id', 'mpd', 'title', 'political_leanings',
       'price', 'npd_year', 'place_wqid', 'place', 'hcounties', 'avg_price',
       'year', 'text', 'political_leaning', 'gender_victims', 'age_victims',
       'count_victims', 'type_accident', 'industrial_accident',
       'transport_accident', 'location_precision', 'first_person', 'anger',
       'disgust', 'fear', 'sadness', 'anticipation', 'surprise', 'joy',
       'trust', 'body_parts', 'num_body_parts', 'body_neg_score', 'cost',
       'accident_latitude', 'accident_location', 'accident_longitude',
       'distance_accident', 'latitude', 'longitude', 'sentences_with_toponyms',
       'toponym_accident', 'predicted_accident'],
      dtype='object')

In [10]:
all_df = all_df[['classification_id', 'accident_latitude', 'accident_longitude', 'toponym_accident', 'distance_accident', 'count_victims', 'num_body_parts']]
all_df.columns = ['classification_id', 'lat', 'lon', 'loc', 'dist', 'n_victims', 'n_body_parts']
all_df['n_victims'] = all_df['n_victims'].fillna('None')
all_df['n_body_parts'] = all_df['n_body_parts'].fillna(0)
all_df.head(2)

,classification_id,lat,lon,loc,dist,n_victims,n_body_parts
0,196046053,53.659500,-1.5566,Horbury,81.128116,None,0
1,188806186,53.744444,-0.3325,Hull,0.000000,None,0


In [11]:
def agg_task_values(task):
    values = [value for value in task if is_value_valid(value)]
    return [value.strip() if isinstance(value, str) else value for value in values]


def is_value_valid(value):
    return value not in ["", "None"] and not pd.isna(value)


def values_with_counts(value):
    if not value:
        return None

    values = [v.strip().casefold() for v in value]
    counter = Counter(values)

    return counter


annotations_df = project.annotations_flattened
annotations_df = annotations_df[annotations_df["workflow_id"].isin(projects_workflows)]
annotations_df = annotations_df.merge(all_df, how="left", left_index=True, right_on="classification_id")
annotations_df = annotations_df.merge(
    subjects[["subject_id", "sid"]],
    how="left",
    left_on="subject_ids",
    right_on="subject_id",
)
annotations_df = annotations_df.drop(
    columns=["classification_id", "workflow_version", "subject_ids", "subject_id"]
)
annotations_df = (
    annotations_df.groupby(["workflow_id", "sid"]).agg(agg_task_values).reset_index()
)
for idx in range(0, 7):
    annotations_df[f"T{idx}"] = annotations_df[f"T{idx}"].apply(values_with_counts)
annotations_df = annotations_df.dropna(
    subset=["T0", "T1", "T2", "T3", "T4", "T5", "T6"], how="all"
)

for col in ['lat', 'lon', 'loc', 'dist', 'n_victims', 'n_body_parts']:
    annotations_df[col] = annotations_df[col].apply(lambda x: set(x).pop() if len(x) > 0 else None)

annotations_df.to_json(os.path.join(out_path, "annotations.json"), orient="records")
annotations_df.head()

,workflow_id,sid,T0,T1,T2,T3,T4,T5,T6,lat,lon,loc,dist,n_victims,n_body_parts
1,18831,31,None,{'male': 72},{'one': 72},{'adult': 72},None,None,None,NaN,NaN,None,NaN,None,NaN
2,18831,42,None,{'male': 72},{'one': 72},{'57': 72},None,None,None,NaN,NaN,None,NaN,None,NaN
5,18831,70,None,None,{'more than one': 72},None,{'all male': 72},None,"{'adults (the default)': 48, 'mixed age groups...",NaN,NaN,None,NaN,None,NaN
8,18831,93,None,{'male': 120},"{'one': 120, 'more than one': 8}",{'adult': 120},{'majority male group': 8},None,{'adults (the default)': 8},NaN,NaN,None,NaN,None,NaN
9,18831,102,None,None,{'more than one': 8},None,{'majority male group': 8},None,"{'adults (the default)': 4, 'mixed age groups'...",NaN,NaN,None,NaN,None,NaN


## Newspapers

In [12]:
psd_regex = re.compile(r"(£(?P<p>\d+))?\s*((?P<s>\d+)s)?\s*((?P<d>\d+\.?\d*)d)?")


def get_political_group(value: str) -> str:
    """
    Get the political group from a political value.
    """
    group = source_politics_df[source_politics_df["label_orig"] == value]["label_simpl"]
    if group:
        return group[0]

    return ""


def convert_price(values: list) -> float:
    """
    Convert a list of price values in £/s/d format to decimal £.

    return: float
        the first value in the list converted to decimal £.
    """
    pounds = []

    for value in values:
        if not value:
            return 0.0

        value = re.sub("\s?½\s?", ".5", value)
        match = psd_regex.search(value)
        p = float(match.group("p") or 0)
        s = float(match.group("s") or 0)
        d = float(match.group("d") or 0) / 12

        pounds_decimal = round(p + (s + d) / 20, 3)

        pounds.append(pounds_decimal)

    return pounds[0]


source_df = pd.read_csv(os.path.join(ext_path, "MPD_export_1846_1920_20230217.csv"))
source_links_df = pd.read_csv(
    os.path.join(ext_path, "newspapers_links.csv"),
    dtype=dict(county="string", historiccounty="string"),
)
source_politics_df = pd.read_csv(os.path.join(ext_path, "politics.csv"))

politics_groups = (
    source_politics_df[["label_orig", "label_simpl"]]
    .drop_duplicates()
    .set_index("label_orig")
    .to_dict()["label_simpl"]
)

source_df = source_df.merge(
    source_links_df[["NLP", "link_to_mpd"]],
    how="inner",
    left_on="id",
    right_on="link_to_mpd",
)
source_df = source_df.drop_duplicates()
# there are multiple columns with the same name
source_df = source_df.drop("county", axis=1)

source_df = source_df.rename(columns=str.lower)
source_df = source_df.rename(
    columns={
        "s-pol": "politics_original",
        "s-title": "title",
        "s-price": "price_original",
    }
)
source_df = source_df[
    [
        "id",
        "chain_id",
        "nlp",
        "title",
        "county",
        "year",
        "politics_original",
        "price_original",
    ]
]

source_df["politics_original"] = source_df["politics_original"].fillna("")
source_df["politics"] = source_df["politics_original"].apply(
    lambda x: politics_groups.get(x, "")
)

source_df["price_original"] = (
    source_df["price_original"].astype(str).apply(lambda x: x.split("<SEP>"))
)
source_df["price"] = source_df["price_original"].apply(convert_price)
source_df["price_range"] = pd.qcut(
    source_df["price"], 4, duplicates="drop", labels=["0-4p", "5p-8p", "8p-£2"]
)

source_df["title"] = source_df["title"].str.strip()

source_df.to_json(os.path.join(out_path, "newspapers.json"), orient="records")

source_df.head()

,id,chain_id,nlp,title,county,year,politics_original,price_original,politics,price,price_range
0,MPD_1846_65,CID_000001,78,LLOYD'S WEEKLY LONDON NEWSPAPER .,london,1846,democratic; anti-poor-law,[3d],liberal,0.013,8p-£2
4,MPD_1847_73,CID_000001,78,LLOYD'S WEEKLY LONDON NEWSPAPER .,london,1847,democratic; anti-poor-law,[3d],liberal,0.013,8p-£2
6,MPD_1847_73,CID_000001,79,LLOYD'S WEEKLY LONDON NEWSPAPER .,london,1847,democratic; anti-poor-law,[3d],liberal,0.013,8p-£2
7,MPD_1851_74,CID_000001,79,LLOYD'S WEEKLY LONDON NEWSPAPER .,london,1851,democratic,[3d],liberal,0.013,8p-£2
11,MPD_1856_97,CID_000001,79,LLOYD'S WEEKLY LONDON NEWSPAPER .,london,1856,democratic; anti-poor-law,"[2d, 3d]",liberal,0.008,5p-8p


In [13]:
source_df["price_range"].unique()

['8p-£2', '5p-8p', '0-4p']
Categories (3, object): ['0-4p' < '5p-8p' < '8p-£2']